# Instalacion de dependencias:

In [ ]:
import pandas as pd
from google.colab import drive
import random
import seaborn as sn
import matplotlib.pyplot as plt
from google.colab import data_table
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
drive.mount('/content/drive')

# Cargado y división del conjunto de tweets

In [ ]:
#@title Cargar conjunto de tweets: { display-mode: "form" }
#@markdown **WS_PATH**: Ruta del directorio del conjunto de datos de tweets a cargar

WS_PATH ="/content/drive/Tweets.csv" #@param {type:"string"}
df = pd.read_csv(WS_PATH)

In [ ]:
print(df['class'].value_counts())
df['class'].replace({'suicide': 0, 'non-suicide':1},inplace=True)
df.drop("Unnamed: 0",axis = 1,inplace=True)
df

Generamos un subset del dataframe con elementos de ambas clases

In [ ]:
random_df= df.sample(n=16037,random_state=50)
random_df['class'].value_counts()

Eliminamos el subset del dataframe original

In [ ]:
df.drop(random_df.index,inplace=True)
df.reset_index(inplace=True, drop=True)
random_df.reset_index(inplace=True, drop=True)
random_df

Guardamos conjunto de datos para explorar con elementos de ambas clases.

In [ ]:
random_df.to_csv('/content/drive/tweets_to_explore.csv',columns={'text','class'},index=False)

Generamos el conjunto de datos que contiene todos los tweets con sentimiento depresivo.

In [ ]:
df_suicides = df[df['class']==0]
df_suicides.reset_index(inplace=True, drop=True)
df_suicides

Guardamos el conjunto:

In [ ]:
df_suicides.to_csv('/content/drive/depression_tweets.csv',columns={'text'},index=False)

# Clasificación de tweets

Cargamos los dos conjuntos ya calculados:

In [ ]:
WS_PATH = '/content/drive/Shareddrives/TFG_2021/final code/SMBC_EXPLORER/'
df_test =pd.read_csv(WS_PATH + 'tweets_to_explore.csv')
df_suicides = pd.read_csv(WS_PATH + 'depression_tweets.csv')

Usamos el SBMC para calcular las similitudes entre ambos conjuntos y cargamos los resultados:

In [ ]:
#@title  { display-mode: "form" }
MATRIX_FILE_NAME = "similarity with depression tweets.pkl" #@param {type:"string"}

try:
  matriz_similitud = pd.read_pickle(WS_PATH +'/similarity_matrix/'+ MATRIX_FILE_NAME)
except Exception as e:
  print("Hay algún problema con el fichero indicado.")
  raise e
else:
  print("Matriz de similitud cargada correctamente.")

data_table.DataTable(matriz_similitud, include_index=True, num_rows_per_page=5)

Si alguno del conjunto a explorar se parece mas de un 60% a alguno del conjunto de depresivos, se clasifica como depresivo. Calculamos la matriz de confusión:

In [ ]:
threshold = 0.6
y_pred = [0 if cosenos[0] >= threshold else 1 for cosenos in matriz_similitud['cosenos_compare']]
data = {'y_Actual':   df_test['class'].tolist(),'y_Predicted': y_pred}

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix, annot=True,fmt="d")
plt.show()

Vamos a observar los valores de precision,recall y f1 para este clasificador:

In [ ]:
target_names = ['Depressive', 'Non-Depressive']
print(classification_report(df['y_Actual'],df['y_Predicted'],target_names=target_names,digits=3))